# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
pt1 = pd.read_csv('city_temperature_df.csv')
pt1

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,2126018,De-kastri,51.4667,140.7833,-9.63,94,98,0.34,RU,1613256586
1,2163355,Hobart,-42.8794,147.3294,17.78,67,20,1.54,AU,1613256342
2,1489656,Teya,60.3778,92.6267,-26.65,85,100,1.71,RU,1613256586
3,2134814,Sola,-13.8833,167.5500,27.19,83,100,8.12,VU,1613256586
4,2092164,Lorengau,-2.0226,147.2712,28.38,75,99,2.76,PG,1613256586
...,...,...,...,...,...,...,...,...,...,...
541,3430443,Necochea,-38.5473,-58.7368,20.00,89,100,1.34,AR,1613256698
542,3038333,Ajaccio,42.0000,8.9167,6.00,70,90,4.63,FR,1613256698
543,2122104,Petropavlovsk-kamchatskiy,53.0452,158.6483,-17.00,66,86,4.00,RU,1613256460
544,1783873,Zhumadian,32.9794,114.0294,10.82,86,99,2.18,CN,1613256657


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

coordinates = pt1[['Lat', 'Lng']]
humidity = pt1['Humidity'].astype('float')

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal = pt1[(pt1['Max Temp'] <= 30) & (pt1['Wind Speed'] > 7) & (pt1['Cloudiness'] < 71)].reset_index(drop=True)
coord2 = ideal[['Lat', 'Lng']]

ideal

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,3372707,Ribeira grande,38.5167,-28.7000,14.37,63,61,12.44,PT,1613256587
1,4020109,Atuona,-9.8000,-139.0333,26.19,79,6,7.60,PF,1613256595
2,2206874,Waipawa,-41.4122,175.5153,20.00,64,20,9.77,NZ,1613256596
3,3838859,Rio gallegos,-51.6226,-69.2181,14.00,33,40,7.72,AR,1613256369
4,3883457,Lebu,-37.6167,-73.6500,14.65,77,3,7.67,CL,1613256589
5,935215,Saint-philippe,-21.3585,55.7679,26.00,73,34,7.20,RE,1613256602
6,3175784,Grosseto,42.8333,11.2500,0.00,51,0,8.23,IT,1613256605
7,3372964,Lagoa,39.0500,-27.9833,13.88,65,42,13.84,PT,1613256604
8,3372472,Vila franca do campo,37.7167,-25.4333,15.56,94,20,8.23,PT,1613256625
9,53654,Mogadishu,2.0371,45.3438,24.12,84,67,8.81,SO,1613256641


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
from pprint import pprint

hotel_df = ideal
h_df = []

target_coordinates = ''
target_radius = 5000
target_type = 'hotel'

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

for index,rows in ideal.iterrows():
    target_coordinates = str(rows['Lat']) + ',' + str(rows['Lng'])
    params = {
        'location': target_coordinates,
        'radius': target_radius,
        'type': target_type,
        'key': g_key
    }
    cityname = rows['City']
    print(f'Printing results for index {index}: {cityname}')
    response = requests.get(base_url, params=params).json()
    
    results = response['results'][0]['name']
    
    try:
        h_df.append(results)
        print(f'Closest {cityname}"s hotel is {results}.')
        
    except (KeyError, IndexError):
            print('Missing info')
print('----------------------')

hotel_df['Hotel'] = h_df

hotel_df.head()


    

Printing results for index 0: Ribeira grande
Closest Ribeira grande"s hotel is Horta.
Printing results for index 1: Atuona
Closest Atuona"s hotel is Atuona.
Printing results for index 2: Waipawa
Closest Waipawa"s hotel is Tora.
Printing results for index 3: Rio gallegos
Closest Rio gallegos"s hotel is Rio Gallegos.
Printing results for index 4: Lebu
Closest Lebu"s hotel is Lebu.
Printing results for index 5: Saint-philippe
Closest Saint-philippe"s hotel is Saint-Philippe.
Printing results for index 6: Grosseto
Closest Grosseto"s hotel is Marrucheti.
Printing results for index 7: Lagoa
Closest Lagoa"s hotel is Santa Cruz Da Graciosa.
Printing results for index 8: Vila franca do campo
Closest Vila franca do campo"s hotel is Vila Franca do Campo.
Printing results for index 9: Mogadishu
Closest Mogadishu"s hotel is Mogadishu.
Printing results for index 10: Bathsheba
Closest Bathsheba"s hotel is Wilson Hill.
Printing results for index 11: The valley
Closest The valley"s hotel is The Valley.

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,3372707,Ribeira grande,38.5167,-28.7000,14.37,63,61,12.44,PT,1613256587,Horta
1,4020109,Atuona,-9.8000,-139.0333,26.19,79,6,7.60,PF,1613256595,Atuona
2,2206874,Waipawa,-41.4122,175.5153,20.00,64,20,9.77,NZ,1613256596,Tora
3,3838859,Rio gallegos,-51.6226,-69.2181,14.00,33,40,7.72,AR,1613256369,Rio Gallegos
4,3883457,Lebu,-37.6167,-73.6500,14.65,77,3,7.67,CL,1613256589,Lebu


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))